Imports and function definitions

In [15]:
import os
from glob import glob
from natsort import natsorted
from nimlab import connectomics as cs
from nimlab.connectomics import ConnectomeSubject
from nimlab.jax_functions import load_image, NiftiMasker
import numpy as np
from tqdm import tqdm

if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

def create_connectome_subjects(cs_roi, cs_brain, roi_list, masker_list, roi_connectome_type, brain_connectome_type, same_connectome, warning_flag):
    # Load paths to all ROI connectome files
    roi_connectome_files_norms = natsorted(glob(os.path.join(cs_roi, "*_norms.npy")))
    roi_connectome_files = [
        (glob(f.split("_norms")[0] + ".npy")[0], f) for f in roi_connectome_files_norms
    ]

    # Load paths to all Brain connectome files
    brain_connectome_files_norms = natsorted(glob(os.path.join(cs_brain, "*_norms.npy")))
    brain_connectome_files = [
        (glob(f.split("_norms")[0] + ".npy")[0], f)
        for f in brain_connectome_files_norms
    ]

    # Create ConnectomeSubject objects
    subjects = []
    for roi_connectome_file, brain_connectome_file in zip(roi_connectome_files, brain_connectome_files):
        subject = ConnectomeSubject(
            roi_connectome_file,
            brain_connectome_file,
            roi_list,
            roi_connectome_type,
            brain_connectome_type,
            same_connectome,
            warning_flag
        )
        subjects.append(subject)
    
    return subjects


Save BOLD signal in the brain ROI as numpy arrays (probably won't be very useful, but just in case)

In [16]:
cs_roi = '/data/nimlab/connectome_npy/GSP1000_MF'
cs_brain = '/data/nimlab/connectome_npy/GSP1000_MF'
masker_list = [] 
roi_connectome_type = 'volume'
brain_connectome_type = 'volume'
same_connectome = False
warning_flag = True
roi_list = {
    1:np.atleast_2d(NiftiMasker().transform('/PHShome/jt041/projects/connectome_differences/rois/1-r_01_gsp1000MF_vs_yeo1000_csf_brainstem_roi.nii.gz')),
    2:np.atleast_2d(NiftiMasker().transform('/PHShome/jt041/projects/connectome_differences/rois/1-r_01_gsp1000MF_vs_yeo1000_masked_brainstem_roi.nii.gz')),
    3:np.atleast_2d(NiftiMasker().transform('/PHShome/jt041/projects/connectome_differences/rois/1-r_01_gsp1000MF_vs_yeo1000_dil_brainstem_roi.nii.gz')),
} 

subjects = create_connectome_subjects(cs_roi, cs_brain, roi_list, masker_list, roi_connectome_type, brain_connectome_type, same_connectome, warning_flag)

# result = cs.calculate_roi_matrix(
#     subjects=subjects,
#     num_workers=1
# )

if not os.path.exists('bold_signal/GSP1000_MF_bold_brainstem_masked_240.npy'):

    results_120 = []
    results_240 = []
    for i, sub in enumerate(tqdm(subjects, desc="Processing subjects")):
        result = cs.extract_avg_signal(
            connectome_mat=np.load(sub.brain_connectome_file),
            roi_mat=np.atleast_2d(NiftiMasker().transform('/PHShome/jt041/projects/connectome_differences/rois/1-r_01_gsp1000MF_vs_yeo1000_masked_brainstem_roi.nii.gz')),
        )
        if result.shape == (240,):
            results_240.append(result)
        if result.shape == (120,):
            results_120.append(result)

    results_240 = np.array(results_240)
    results_120 = np.array(results_120)

    np.save('bold_signal/GSP1000_MF_bold_brainstem_masked_240.npy', results_240)
    np.save('bold_signal/GSP1000_MF_bold_brainstem_masked_120.npy', results_120)


